## Import / Config

In [109]:
import pandas as pd
from anyio import current_time
from dns.e164 import query
from sqlalchemy import create_engine, text
from neo4j import GraphDatabase
import cred_pg as c
import cred_neo4j as cc

## Engine

In [110]:
engine_postgres = create_engine(
    f'postgresql+psycopg://{c.pg_userid}:{c.pg_password}@{c.pg_host}/{c.pg_db}',
    connect_args={'options': '-c search_path=$user,ugeobln,umisc,umobility,usozmed,public', 'keepalives_idle': 120},
    pool_size=1,
    max_overflow=0,
    execution_options={'isolation_level': 'AUTOCOMMIT'}
)

engine_neo4j = GraphDatabase.driver(cc.neo4j_host, auth=(cc.neo4j_userid, cc.neo4j_password))


In [111]:
# engine.dispose()

## Code

In [178]:
def get_closest_station(lat_lng):
    with engine_postgres.connect() as con:
        query = f"""
            SELECT hid, bez,
                ST_Distance(
                pos,
                'SRID=4326;POINT({lat_lng[1]} {lat_lng[0]})'::geography
            ) AS distanz_in_meter
        FROM haltestelle
        ORDER BY distanz_in_meter ASC
        LIMIT 1;
        """
        df = pd.read_sql_query(text(query), con)
        return df


def get_path_start_finish(station_start, station_ziel):
    with engine_neo4j.session() as session:
        query = f"""
            MATCH (h1:Haltestelle {{bez: '{station_start.iloc[0]["bez"]}'}}), (h2:Haltestelle {{bez: '{station_ziel.iloc[0]["bez"]}'}})
            CALL gds.shortestPath.dijkstra.stream('bubahn', {{
                sourceNode: h1,
                targetNode: h2
                }})
            YIELD index, sourceNode, targetNode, nodeIds
            with [nodeId IN nodeIds | gds.util.asNode(nodeId)] AS p
            UNWIND p as station
            RETURN station.bez as bez, station.hid as hid;
        """
        res = session.run(query)
        return pd.DataFrame(res.data())


def get_seg_between_stations(station_a_id, station_b_id):
    with engine_postgres.connect() as con:
        query = f"""
            SELECT uu.ulid, ul.bez
            FROM umobility.abschnitt AS ua
                INNER JOIN umobility.unterlinie AS uu ON ua.ulid = uu.ulid
                INNER JOIN umobility.linie AS ul ON uu.lid = ul.lid
            WHERE hid_a = '{station_a_id}'
                AND hid_b = '{station_b_id}';
        """
        df = pd.read_sql_query(text(query), con)
        return df


def get_departure_time(station_id, time, ulid, date):
    print(f"--get_departure_time({station_id}, {time})")
    with engine_postgres.connect() as con:
        query = f"""
            SELECT halt.zeit_abfahrt as zeit
            FROM umobility.zeitplan AS zeitplan
                INNER JOIN umobility.fahrt AS fahrt ON zeitplan.zpid = fahrt.zpid
                INNER JOIN umobility.halt AS halt ON fahrt.fid = halt.fid
            WHERE ('{date}' BETWEEN zeitplan.datum_beginn AND zeitplan.datum_ende)
                AND halt.hid = '{station_id}'
                AND halt.zeit_abfahrt > '{time}'
                AND fahrt.ulid = {ulid}
            ORDER BY halt.zeit_abfahrt ASC
            LIMIT 1;
        """
        df = pd.read_sql_query(text(query), con)
        print(f"{query}")
        print(f"--{df}")
        return df.iloc[0]["zeit"]


def get_arrival_time(station_id, time, ulid, date):
    print(f"--get_arrival_time({station_id}, {time})")
    with engine_postgres.connect() as con:
        query = f"""
            SELECT halt.zeit_ankunft as zeit
            FROM umobility.zeitplan AS zeitplan
                INNER JOIN umobility.fahrt AS fahrt ON zeitplan.zpid = fahrt.zpid
                INNER JOIN umobility.halt AS halt ON fahrt.fid = halt.fid
            WHERE ('{date}' BETWEEN zeitplan.datum_beginn AND zeitplan.datum_ende)
                AND halt.hid = '{station_id}'
                AND halt.zeit_ankunft > '{time}'
                AND fahrt.ulid = {ulid}
            ORDER BY halt.zeit_ankunft ASC
            LIMIT 1;
        """
        df = pd.read_sql_query(text(query), con)
        print(f"{query}")
        print(f"--{df}")
        return df.iloc[0]["zeit"]
    
    
def fahrverbindung(lat_lng_start, lat_lng_ziel, startdatum, startzeit):
    station_start = get_closest_station(lat_lng_start)
    print(f"Starthaltestelle\n{station_start}")

    station_ziel = get_closest_station(lat_lng_ziel)
    print(f"\nZielhaltestelle\n{station_ziel}")

    pfad = get_path_start_finish(station_start, station_ziel)

    results = []

    line_prev = ""
    time_current = ""
    for i in range(len(pfad) - 1):
        line_info = get_seg_between_stations(pfad.iloc[i]["hid"], pfad.iloc[i + 1]["hid"])
        line = line_info.iloc[0]["bez"]
        line_ulid = line_info.iloc[0]["ulid"]
        
        print(line_ulid)
        
        if line_prev != "":
            if line_prev == line:
                umstieg = "nein"
            else:
                umstieg = "ja"
        else:
            umstieg = "nein"

        if time_current == "":
            time_current = startzeit
        
        time_departure = get_departure_time(pfad.iloc[i]["hid"], time_current, line_ulid, startdatum)
        time_current = time_departure
        time_arrival = get_arrival_time(pfad.iloc[i + 1 ]["hid"], time_current, line_ulid, startdatum)
        time_current = time_arrival
            

        table_entry = {
            "hid1": f"{pfad.iloc[i]["hid"]}",
            "hid2": f"{pfad.iloc[i + 1]["hid"]}",
            "von": f"{pfad.iloc[i]["bez"]}",
            "nach": f"{pfad.iloc[i + 1]["bez"]}",
            "U": f"{line}",
            "Abfahrt": f"{time_departure}",
            "Ankunft": f"{time_arrival}",
            "Umstieg": f"{umstieg}",
        }

        results.append(table_entry)

    df_result = pd.DataFrame(results)
    print(f"\nVerbindung\n{df_result}")

    return

## Ausführung

### Eberbacher Str. 1 nach Meierottostraße 10

In [179]:
fahrverbindung([52.473176, 13.313740], [52.4969134, 13.327166], "2021-04-01", "11:00:00")

Starthaltestelle
     hid               bez  distanz_in_meter
0  10291  RüdesheimerPlatz         90.804351

Zielhaltestelle
     hid          bez  distanz_in_meter
0  10303  SpichernStr        236.990577
1019002
--get_departure_time(10291, 11:00:00)

            SELECT halt.zeit_abfahrt as zeit
FROM umobility.zeitplan AS zeitplan
    INNER JOIN umobility.fahrt AS fahrt ON zeitplan.zpid = fahrt.zpid
    INNER JOIN umobility.halt AS halt ON fahrt.fid = halt.fid
WHERE ('2021-04-01' BETWEEN zeitplan.datum_beginn AND zeitplan.datum_ende)
    AND halt.hid = '10291'
    AND halt.zeit_abfahrt > '11:00:00'
    AND fahrt.ulid = 1019002
ORDER BY halt.zeit_abfahrt ASC
LIMIT 1;
        
--       zeit
0  11:00:46
--get_arrival_time(10152, 11:00:46)

            SELECT halt.zeit_ankunft as zeit
FROM umobility.zeitplan AS zeitplan
    INNER JOIN umobility.fahrt AS fahrt ON zeitplan.zpid = fahrt.zpid
    INNER JOIN umobility.halt AS halt ON fahrt.fid = halt.fid
WHERE ('2021-04-01' BETWEEN zeitplan.datu

### Eberbacher Str. 1 nach Ballenstedter Str. 6

In [114]:
fahrverbindung([52.473176, 13.313740], [52.494833, 13.3038814], "2021-04-01", "11:00:00")

Starthaltestelle
     hid               bez  distanz_in_meter
0  10291  RüdesheimerPlatz         90.804351

Zielhaltestelle
     hid            bez  distanz_in_meter
0  10242  KonstanzerStr         444.20619
10291 | 10152 | RüdesheimerPlatz | HeidelbergerPlatz | U3 | 11:00:46 | 11:05:16 | nein
10152 | 10209 | HeidelbergerPlatz | FehrbellinerPlatz | U3 | 11:07:09 | 11:09:22 | nein
10209 | 10242 | FehrbellinerPlatz | KonstanzerStr | U7 | 11:05:31 | 11:07:18 | nein


### Eberbacher Str. 1 nach Elsastraße 2

In [115]:
fahrverbindung([52.473176, 13.313740], [52.4752505, 13.3308319], "2021-04-01", "11:00:00")

Starthaltestelle
     hid               bez  distanz_in_meter
0  10291  RüdesheimerPlatz         90.804351

Zielhaltestelle
     hid          bez  distanz_in_meter
0  10148  Bundesplatz        261.707791
10291 | 10152 | RüdesheimerPlatz | HeidelbergerPlatz | U3 | 11:00:46 | 11:05:16 | nein
10152 | 10209 | HeidelbergerPlatz | FehrbellinerPlatz | U3 | 11:07:09 | 11:09:22 | nein
10209 | 10195 | FehrbellinerPlatz | BlisseStr | U7 | 11:05:31 | 11:07:18 | nein
10195 | 10189 | BlisseStr | BerlinerStr | U7 | 11:06:40 | 11:06:50 | nein
10189 | 10148 | BerlinerStr | Bundesplatz | U9 | 11:06:00 | 11:06:04 | nein
